### Camera

In [1]:
from jetracer.camera_utils import JetRacerCamera, release_cam
import time

release_cam()
time.sleep(2)

camera = JetRacerCamera('training')
camera.start()


In [2]:
import cv2
import time

cap = cv2.VideoCapture(0)

if cap.isOpened():
    for i in range(3):
        ret, frame = cap.read()
        if ret:
            cv2.imwrite(f"test_frame_{i}.jpg", frame)
        time.sleep(1)
    cap.release()


In [3]:
import cv2
import time

class AutonomousRacecarCamera:
    def __init__(self, mode='inference', width=640, height=480, fps=21):
        self.mode = mode
        self.width = width
        self.height = height
        self.fps = fps
        self.output_size = (224, 224) if mode in ['inference', 'training'] else (width, height)
        self.camera = None
        self.running = False
        self.last_frame = None

    def start(self):
        self.camera = cv2.VideoCapture(0)
        if self.camera.isOpened():
            ret, frame = self.camera.read()
            if ret:
                self.running = True
                return True
        return False

    def read(self):
        if not self.running:
            return self.last_frame
        ret, frame = self.camera.read()
        if ret:
            frame = cv2.resize(frame, self.output_size)
            self.last_frame = frame
            return frame
        return self.last_frame

    def stop(self):
        self.running = False
        if self.camera:
            self.camera.release()

    @property
    def value(self):
        return self.read()


### Data Collection

In [4]:
import cv2
import ipywidgets
import threading
import time
import os
import uuid
from IPython.display import display

class ImprovedDataCollection:
    def __init__(self):
        self.base_dir = "/home/checker/Autonomous-Racecar/data/improved_training"
        self.collecting = False
        self.categories = {
            'straight_center': None,
            'straight_left_correct': None,
            'straight_right_correct': None,
            'left_turn_gentle': None,
            'left_turn_sharp': None,
            'left_turn_exit': None,
            'recovery_left': None,
            'recovery_right': None
        }
        for c in self.categories:
            os.makedirs(f"{self.base_dir}/{c}", exist_ok=True)
        self.counts = {c: 0 for c in self.categories}


### Model

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
import time
import numpy as np

torch.cuda.is_available = lambda: False
device = torch.device('cpu')

class StableRacingParams:
    THROTTLE = 0.25
    STEERING_GAIN = 0.6
    MAX_STEERING = 0.8
    MIN_STEERING = -0.8
    UPDATE_RATE = 0.05

def preprocess_image(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return transform(image).unsqueeze(0).to(device)


In [6]:
import os

for root, dirs, files in os.walk('/home/checker'):
    for f in files:
        if f.endswith('.pth'):
            pass


In [7]:
import smbus
import time

class CorrectThrottleTest:
    def __init__(self):
        self.bus = smbus.SMBus(7)
        self.address = 0x40
        self.throttle_channel = 1
        self.center_pulse = 1500
        self.max_forward = 1400
        self.initialize_pca9685()

    def initialize_pca9685(self):
        self.bus.write_byte_data(self.address, 0x00, 0x10)
        time.sleep(0.005)
        prescale = int(25000000 / (4096 * 50) - 1)
        self.bus.write_byte_data(self.address, 0xFE, prescale)
        self.bus.write_byte_data(self.address, 0x00, 0x20)
        time.sleep(0.005)
        self.bus.write_byte_data(self.address, 0x01, 0x04)
        self.set_servo_position(self.throttle_channel, self.center_pulse)

    def set_servo_position(self, channel, pulse_us):
        pwm_val = int((pulse_us * 4096) / 20000)
        base_reg = 0x06 + 4 * channel
        self.bus.write_byte_data(self.address, base_reg, 0)
        self.bus.write_byte_data(self.address, base_reg + 1, 0)
        self.bus.write_byte_data(self.address, base_reg + 2, pwm_val & 0xFF)
        self.bus.write_byte_data(self.address, base_reg + 3, (pwm_val >> 8) & 0xFF)

    def test_correct_range(self):
        for speed in [1480, 1460, 1440, 1420, 1400]:
            self.set_servo_position(self.throttle_channel, speed)
            time.sleep(3)
            self.set_servo_position(self.throttle_channel, 1500)
            time.sleep(1)
